In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, SimpleRNN
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import re

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

2024-05-05 04:25:22.426352: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-05 04:25:22.426443: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-05 04:25:22.597599: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
data = pd.read_csv('../input/final-sarcasm-dataset/Dataset.csv')
print(data.columns)
data = data[['Text','Sarcasm']]
data.head()

Index(['Comments', 'Emojis', 'Text', 'Sarcasm'], dtype='object')


,Text,Sarcasm
0,That's how we roll . . . . . . . . #cinemaddi...,1
1,#Sarcasm,1
2,At my bro Barbara school . I'm only one can i...,1
3,And I don’t know why people get hurt #sarcas...,1
4,Yay. I see a #cold in my near #future #jokes...,1


In [3]:
data['Text'] = data['Text'].astype(str)

data['Text'] = data['Text'].apply(lambda x: x.lower())
data['Text'] = data['Text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

print(data[ data['Sarcasm'] == 1].size)
print(data[ data['Sarcasm'] == 0].size)

data.head()

18520
26140


,Text,Sarcasm
0,thats how we roll cinemaddicts9 talko...,1
1,sarcasm,1
2,at my bro barbara school im only one can in ...,1
3,and i dont know why people get hurt sarcasm ...,1
4,yay i see a cold in my near future jokes fun...,1


In [4]:
max_features = 2000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(data['Text'].values)
X = tokenizer.texts_to_sequences(data['Text'].values)
X = pad_sequences(X)

In [5]:
embed_dim = 128
rnn_out = 512

model = Sequential()
model.add(Embedding(max_features, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(SimpleRNN(rnn_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
model.build((None, X.shape[1]))
print(model.summary())

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:86: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 686, 128)       │       256,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d               │ (None, 686, 128)       │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ (None, 512)            │       328,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 2)              │         1,026 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 585,218 (2.23 MB)

 Trainable params: 585,218 (2.23 MB)

 Non-trainable params: 0 (0.00 B)

None


In [6]:
Y = pd.get_dummies(data['Sarcasm']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(14961, 686) (14961, 2)
(7369, 686) (7369, 2)


In [9]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 10, batch_size=batch_size, verbose = 2)

Epoch 1/10
468/468 - 42s - 90ms/step - accuracy: 0.8689 - loss: 0.3112
Epoch 2/10
468/468 - 42s - 90ms/step - accuracy: 0.8743 - loss: 0.2992
Epoch 3/10
468/468 - 42s - 89ms/step - accuracy: 0.8821 - loss: 0.2819
Epoch 4/10
468/468 - 42s - 89ms/step - accuracy: 0.8862 - loss: 0.2736
Epoch 5/10
468/468 - 42s - 89ms/step - accuracy: 0.8812 - loss: 0.2830
Epoch 6/10
468/468 - 42s - 89ms/step - accuracy: 0.8906 - loss: 0.2595
Epoch 7/10
468/468 - 42s - 89ms/step - accuracy: 0.8959 - loss: 0.2473
Epoch 8/10
468/468 - 42s - 89ms/step - accuracy: 0.8961 - loss: 0.2439
Epoch 9/10
468/468 - 42s - 89ms/step - accuracy: 0.9009 - loss: 0.2350
Epoch 10/10
468/468 - 42s - 89ms/step - accuracy: 0.8798 - loss: 0.2829


In [10]:
model.save("Models/RNN_only_text_updated.h5")

In [11]:
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

W0000 00:00:1714883856.574741     117 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


184/184 - 5s - 29ms/step - accuracy: 0.8405 - loss: 0.4500
score: 0.45
acc: 0.84


In [12]:
pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
for x in range(len(X_validate)):
    
    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose=0)[0]
   
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1
       
    if np.argmax(Y_validate[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1



print("pos_acc", pos_correct/pos_cnt*100, "%")
print("neg_acc", neg_correct/neg_cnt*100, "%")

pos_acc 65.51155115511551 %
neg_acc 98.65771812080537 %


In [13]:
twt = ['Meetings: Because none of us is as dumb as all of us.']
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=28, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0  108 1430   10  118    9   56  664   56   20   10  118]]
1/1 - 0s - 470ms/step
negative
